# Quantitative Methods in Finance

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantMethodsFinance/blob/main/NOTEBOOKS/2_Interest_rates_basics.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.

Clone the repository

In [ ]:
from IPython.display import clear_output

!rm -r {'QuantMethodsFinance'}

!git clone https://github.com/Lolillosky/QuantMethodsFinance

import sys
sys.modules.pop
sys.path.insert(0,'QuantMethodsFinance/CODE')

clear_output()

Import some libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import minimize
import pandas as pd
import re
from scipy.interpolate import CubicSpline


Modify PATH variable depending on how you run the notebook

In [ ]:
# To be used if notebook is run in Google Colab
PATH = '/content/QuantMethodsFinance/'
# To be used if notebook is run in a local clone of the repository
PATH = '../'


## Interest rate 

Given the following:

| Concept | Amount | 
|----------|----------|
| Initial investment    | 100     |
| Final value    | 150     |
| T-t (years)    | 5     |

Calculate simple, compounded and continuously compounded interest


In [ ]:
# Your code here

## Continuously compounded interest

Check numerically that:

$$\lim_{n \to \infty}\left(1+r\frac{T-t}{n}\right)^n=\exp\left(r\left(T-t\right)\right)$$

Assume $r=0.03$ $T-t=5$

In [ ]:
# Your code here


## Internal rate of return / yield of a bond 

The following code is a simplified version of the pricing of a bond forgeting about dates and day count conventions.

In [ ]:

from enum import Enum

class freq_options(Enum):
    QUARTERLY = 1
    SEMI_ANNUAL = 2
    ANUAL = 3


def bond_price(TTM, coupon, yield_to_mat, freq):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    return np.sum(coupon * delta * (1.0 + yield_to_mat)**(-payment_times)) + (1.0 + yield_to_mat)**(-TTM) 


In [ ]:
bond_price(5,0.03,0.05,freq_options.ANUAL)

For a 5 year maturity bond, with annual coupons of $3\%$, plot the dirty price as a function of its yield. If the yield of a bond rises, does the bond holder gain or loose money?

In [ ]:
# Your code here

For yields equal to $1\%,\ 3\%,\ 5\%$, plot the dirty price as time goes by and we approach the maturity of the bond. 

In [ ]:
# Your code here

If the price of the bond is $103\%$, compute its yield. Use fsolve function of Scipy.

In [ ]:
# Your code here

## Duration

Code a function to compute the duration of a bond for a given maturity, coupon, yield and payment frequency.

In [ ]:
def bond_duration(TTM, coupon, yield_to_mat, freq):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]

    # your code here

For a 30 year bond with a coupon of $3\%$, plot the bond duration as time evolves for yields of $1\%,\ 3\%,\ 5\%$.

In [ ]:
# Your code here

Same for a zero coupon bond (bond with coupon equal to $0\%$).

In [ ]:
# Your code here

Can you provide a physical interpretation of duration? 

## Bootstrapping the yield curve

Let us read the historical data included in the spgb_hist.csv file.

In [ ]:
spgb_hist = pd.read_csv(PATH + 'DATA/spgb_hist.csv', parse_dates=True, index_col=0, dayfirst=True)

spgb_hist.head()

The following code extracts the maturity and coupon of every bond. We will bootstrap the curve on the last date of the dataframe.

In [ ]:

# Calibration date will be last date of the dataframe.
calibration_date = spgb_hist.index[-1]

# Regular expression pattern to extract dates in the format mm/dd/yyyy.
date_pattern =  r"\b\d*\.?\d{0,2}\s(\d{1,2}/\d{1,2}/\d{4})\b"

maturity_dates = [re.search(date_pattern, item).group(1) for item in spgb_hist.columns if re.search(date_pattern, item)]

maturity_dates = pd.to_datetime(maturity_dates, format='%m/%d/%Y');

# Regular expression pattern to extract coupons in the format d.dd.
coupon_pattern = r"\b\d+(\.\d{1,2})?\b"

coupons = np.float64([re.search(coupon_pattern, item).group() for item in spgb_hist.columns if re.search(coupon_pattern, item)])/100

# Year fraction to maturity of each bond.
years_to_maturity = (maturity_dates - calibration_date).days / 365

# Let us print coupons and years to maturity.
print(coupons)
print(years_to_maturity)



The following function calculates the yield of a bond from its coupon, maturity and price.

In [ ]:
def bond_yield(TTM, coupon, price, freq):

    return fsolve(lambda x: bond_price(TTM, coupon, x, freq) - price, coupon,xtol=1e-8)[0]

Let us check that the function works.

In [ ]:
coupon = 0.05
price = 1.00
maturity = 30

bond_yield(maturity, coupon, price, freq_options.ANUAL)

The following class is the IR curve class. It wrapps a cubic spline interpolator and assumes cubic spline interpolation in zero copupon rates in rates * time (an ussual assumption).

In [ ]:
class IR_Curve:

    def __init__(self, time_pillars, rates):
        # Append zero at the beginning for both time_pillars and rates
        time_pillars = np.concatenate(([0], time_pillars))
        rates = np.concatenate(([0], rates))
        time_times_rate = rates * time_pillars

        # Natural cubic spline interpolation
        self.interpolator = CubicSpline(time_pillars, time_times_rate, bc_type='natural')

    def zero_coupon_rates(self, time_pillars):
        # Check for negative maturities and raise an error if found
        if np.any(time_pillars < 0):
            raise ValueError("Maturity cannot be negative.")

        # Calculate the true limit for zero-coupon rate as t approaches 0
        zero_coupon_rates = np.zeros_like(time_pillars, dtype=float)
        non_zero_mask = time_pillars != 0

        # Calculate rates for non-zero times
        zero_coupon_rates[non_zero_mask] = self.interpolator(time_pillars[non_zero_mask]) / time_pillars[non_zero_mask]
        
        # Use the derivative at t = 0 as the limit for the zero-coupon rate at t = 0
        zero_coupon_rates[~non_zero_mask] = self.interpolator.derivative()(0)

        return zero_coupon_rates

    def discount_factors(self, time_pillars):
        # Check for negative maturities and raise an error if found
        if np.any(time_pillars < 0):
            raise ValueError("Maturity cannot be negative.")
        
        # Evaluate the spline and compute discount factors
        return np.exp(-self.interpolator(time_pillars))



In [ ]:
pillars = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 20, 30])
rates = np.array([0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055])
#rates = np.ones_like(pillars) * (np.log(1.05))
ir_curve = IR_Curve(pillars, rates)

time_pillars = np.linspace(0.0, 30, 100)


plt.plot(time_pillars, ir_curve.zero_coupon_rates(time_pillars))
plt.plot(pillars, rates, 'o', color='red')


Taking into consideration the curve object, modify the bond price function so that it prices bonds discounting its cashflows with the curve.

In [92]:
def bond_price_curve(TTM, coupon, curve, freq):

    if freq == freq_options.QUARTERLY:
          delta = 0.25
    elif freq == freq_options.SEMI_ANNUAL:
        delta = 0.5
    elif freq == freq_options.ANUAL:
        delta = 1
    else:
        raise ValueError("Invalid frequency")

    payment_times = np.arange(TTM,0, step=-delta)[::-1]
  
    # your code here

In [ ]:
bond_price_curve(5, 0.05, ir_curve, freq_options.ANUAL)

Write a function that computes the yield of a bond if the cashflows are discounted with the curve.

In [ ]:
def bond_yield_curve(TTM, coupon, curve, freq):

    bond_target_price = bond_price_curve(TTM, coupon, curve, freq)

    return fsolve(lambda x: bond_price(TTM, coupon, x, freq) - bond_target_price, coupon, xtol=1e-8)[0]

In [ ]:
ttm = np.arange(1,31)

y = [bond_yield_curve(m, 0.05, ir_curve, freq_options.ANUAL) for m in ttm] 

plt.plot(ttm, y)

plt.plot(time_pillars, ir_curve.zero_coupon_rates(time_pillars))
plt.plot(pillars, rates, 'o', color='red')


In [ ]:
class yield_curve_bootsrap:

    def __init__(self, bond_maturities, bond_yields, bond_coupons, curve_pillars, init_guess=0.05):
        
        self.bond_maturities = bond_maturities
        self.bond_yields = bond_yields
        self.curve_pillars = curve_pillars
        self.bond_coupons = np.array(bond_coupons)
        self.init_guess = init_guess*np.ones_like(curve_pillars)

    def compute_model_yields(self, curve_rates):

        ir_curve = IR_Curve(self.curve_pillars, curve_rates)

        return np.array([bond_yield_curve(m, c, ir_curve, freq_options.ANUAL) for m,c in zip(self.bond_maturities, self.bond_coupons)])

    def compute_yield_mse(self, curve_rates):
        
        model_yields = self.compute_model_yields(curve_rates)

        return np.mean((model_yields - self.bond_yields)**2)
    
    def bootstrap(self):
        
        return minimize(self.compute_yield_mse, self.init_guess)['x']
        
    

In [ ]:
pillars = np.array([0.25, 0.5, 1, 2, 3, 5, 7, 10, 15,20,25 ,30, 40,50])
#rates = np.array([0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06])

boot = yield_curve_bootsrap(years_to_maturity, spgb_hist.iloc[-1]/100, coupons, pillars)


#boot.compute_yield_mse(rates)

rates = boot.bootstrap()


In [ ]:
plt.plot(years_to_maturity, boot.compute_model_yields(rates), 'o')
plt.plot(years_to_maturity, spgb_hist.iloc[-1]/100, '.')
plt.plot(np.arra, rates, 'o', color='red')